In [ ]:
pip install scikit-learn==1.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 36.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2


In [ ]:
# @title Instalar bibliotecas

%%capture
!pip install ultralytics

In [ ]:
# @title Importar os módulos

import os
import cv2
import glob
import torch
import shutil
import zipfile
import ultralytics
import numpy as np
import pandas as pd
from torch import nn
from PIL import Image
from time import sleep
from joblib import load
from pydantic import BaseModel
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
from ultralytics.engine.results import Results
from sklearn.metrics import classification_report


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# @title Obter o dataset

PATH = '/content/drive/MyDrive/Materiais do TCC/Teste YOLO/Projeto Canoas/TCC2'
dataset_root = f'{PATH}/keypoints_pose_teste.csv'

df = pd.read_csv(dataset_root)


In [ ]:
# @title Funções

class NeuralNet(nn.Module):
    def __init__(
        self,
        input_size=34,
        hidden_size=512,
        num_classes=5
    ):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out


class KeypointClassification:
    def __init__(self, path_model: str):
        """
        Inicializa a classe KeypointClassification.
        :param path_model: Caminho para o modelo pré-treinado.
        """
        self.path_model = path_model
        # self.classes = ['Dor_Joelho', 'Caminhando', 'Mao_na_cabeca', 'Caido', 'Mao_No_Peito', 'Cansado']
        self.classes = ['Caido', 'Caminhando', 'Cansado', 'Mao_na_cabeca', 'Mao_No_Peito']
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.load_model()

    def load_model(self):
        """
        Carrega o modelo pré-treinado.
        :return: Modelo carregado.
        """
        model = NeuralNet()
        model.load_state_dict(torch.load(self.path_model, map_location=self.device))
        return model

    def __call__(self, input_keypoint):
        """
        Classifica o keypoint de entrada.
        :param input_keypoint: Keypoint a ser classificado.
        :return: Classe prevista para o keypoint.
        """
        if not isinstance(input_keypoint, torch.Tensor):
            input_keypoint = torch.tensor(input_keypoint, dtype=torch.float32)
        out = self.model(input_keypoint)
        _, predict = torch.max(out, -1)
        label_predict = self.classes[predict]
        return label_predict


classification_keypoint = KeypointClassification(PATH + '/pesos_pose_classification_v1.pt')
model_svm = load(f'{PATH}/pesos_pose_classification_v1_svc.joblib')


# Testando MLP

In [ ]:
# @title Previsões originais

y_real_mlp = []
y_pred_mlp = []

for indice, linha in df.iterrows():
    label = linha.iloc[1]
    linha = linha.to_list()[2:]

    results_classification = classification_keypoint(linha)

    y_real_mlp.append(label.lower())
    y_pred_mlp.append(results_classification.lower())

print(classification_report(y_real_mlp, y_pred_mlp))


               precision    recall  f1-score   support

        caido       0.87      0.95      0.91        41
   caminhando       0.93      0.95      0.94        44
      cansado       0.94      1.00      0.97        45
mao_na_cabeca       0.10      0.12      0.11        41
 mao_no_peito       0.00      0.00      0.00        43

     accuracy                           0.61       214
    macro avg       0.57      0.61      0.59       214
 weighted avg       0.58      0.61      0.59       214



In [ ]:
# @title Previsões ajustadas

y_real_ajustadas_mlp = []
y_pred_ajustadas_mlp = []

for indice, linha in df.iterrows():
    label = linha.iloc[1]
    linha = linha.to_list()[2:]

    results_classification = classification_keypoint(linha)

    if results_classification.lower() == 'Mao_na_cabeca'.lower():
        results_classification = 'Mao_No_Peito'.lower()
    elif results_classification.lower() == 'Mao_No_Peito'.lower():
        results_classification = 'Mao_na_cabeca'.lower()

    y_real_ajustadas_mlp.append(label.lower())
    y_pred_ajustadas_mlp.append(results_classification.lower())

print(classification_report(y_real_ajustadas_mlp, y_pred_ajustadas_mlp))


               precision    recall  f1-score   support

        caido       0.87      0.95      0.91        41
   caminhando       0.93      0.95      0.94        44
      cansado       0.94      1.00      0.97        45
mao_na_cabeca       1.00      0.68      0.81        41
 mao_no_peito       0.88      0.98      0.92        43

     accuracy                           0.92       214
    macro avg       0.92      0.91      0.91       214
 weighted avg       0.92      0.92      0.91       214



# Testando SVM

In [ ]:
# @title Previsões originais

def mapear_previsao(previsao):
    nomes = {
        0: 'Caido',
        1: 'Caminhando',
        2: 'Cansado',
        3: 'Mao_No_Peito',
        4: 'Mao_na_cabeca'
    }

    return nomes[previsao]


y_real_svm = []
y_pred_svm = []

for indice, linha in df.iterrows():
    label = linha.iloc[1]
    linha = linha.to_list()[2:]

    previsao = model_svm.predict(np.array([linha]))[0]

    y_real_svm.append(label.lower())
    y_pred_svm.append(mapear_previsao(previsao).lower())

print(classification_report(y_real_svm, y_pred_svm))


               precision    recall  f1-score   support

        caido       0.32      0.88      0.47        41
   caminhando       0.57      0.09      0.16        44
      cansado       1.00      0.07      0.12        45
mao_na_cabeca       1.00      0.10      0.18        41
 mao_no_peito       0.32      0.65      0.43        43

     accuracy                           0.35       214
    macro avg       0.64      0.36      0.27       214
 weighted avg       0.64      0.35      0.27       214

